In [1]:
import xml.etree.ElementTree as ET
import os
import re

In [2]:
PATTERN1 = re.compile(r"(?<=[a-zA-Z])\n")
PATTERN2 = re.compile(r"\s+")

def clean_text(text):
    out = PATTERN1.sub(". ", text)
    return PATTERN2.sub(" ", out)

In [3]:
def load_data(path):
    data = []
    for file in os.listdir(path):
        with open(f"{path}/{file}", "r") as f:
            raw = f.read()
        root = ET.fromstring(raw)
        doc = clean_text(root.find("body").text)
        for summary in root.find("summaries").findall("summary"):
            if summary.attrib["ratio"] == "10":
                kind = summary.attrib["type"]
                text = clean_text(summary.find("body").text)
                data.append((doc, text, kind))
    return data

In [4]:
data = load_data("PSC_1.0/data")

# ROUGE

- **P** - precision
- **R** - recall
- **F** - F-measure

**ROUGE: A Package for Automatic Evaluation of Summaries**   
Chin-Yew Lin

https://www.aclweb.org/anthology/W04-1013.pdf

```
!pip install rouge
```

In [5]:
from rouge import Rouge 

In [6]:
data[0]

('ŚWIĘTO NIEPODLEGŁOŚCI. Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego. Poszerzanie wolności. RYS. ANDRZEJ LEGUS. JERZY BUZEK. Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Jakie pożytki czerpiemy dziś z ponownie odzyskanej wolności? Jaka jest dzisiejsza wymowa słowa patriotyzm? Jak teraz układają się relacje między Polakami a ich państwem? Na czym w 1999 roku polega czyn obywatelski? Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesność przenikają się w stopniu najpełniejszym. Dziedzictwo. 11 listopada to wielkie i radosne święto. Odzyskanie wolności zawsze stanowi tytuł do narodowej dumy. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. 

In [7]:
doc_hash = hash(data[0][0])

In [8]:
references = [s for d, s, _ in data if hash(d) == doc_hash]

In [9]:
len(references)

5

In [10]:
references

['11 listopada to wielkie i radosne święto. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. Pamięć o tych, którzy mimo upokorzeń i klęsk nigdy nie pogodzili się z utratą wolności, a także o tych, którzy ów protest przechowali przez noc nazizmu i komunizmu na zawsze pozostanie źródłem naszej siły. Jak dziś postrzegamy czyn patriotyczny? Dziś narodowy czyn ma swój początek w gminie, gdzie podejmowane są decyzje dotyczące lokalnej społeczności. Kultura narodowa i lokalna są ze sobą splecione. W małych ojczyznach dobrze zagospodarowano wolność. właśnie tam najsilniejsza jest owa pamięć historyczna. W czasach globalizacji nowoczesny patriotyzm rozumiem właśnie jako budowanie Polski na dole, w lokalnej przestrzeni duchowej. czyn obywatelski jest rozumiany jako aktywność społeczna, a nie jako reakcja na zagrożenia zewnętrzne. Jest to także czyn radosny.',
 'Niepodległość nieprzypadkowo kojarzona jest dziś z nazwiskami Piłsudskiego, Dmowskiego, Paderewskiego. One bow

In [11]:
summary = ("Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego. "
           "Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji "
           "nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie "
           "wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Pamięć "
           "o tych, którzy mimo 123 lat niewoli, upokorzeń i klęsk, nigdy nie pogodzili się z utratą "
           "wolności, a także o tych, którzy ów protest przechowali przez noc nazizmu i komunizmu na "
           "zawsze pozostanie źródłem naszej siły. Kraków jest dziś przykładem tego, jak działania "
           "na rzecz ochrony narodowego dziedzictwa wpływają na rozwój miasta i warunki życia jego "
           "mieszkańców. Było to możliwe także dlatego, że właśnie tam najsilniejsza jest owa pamięć "
           "historyczna, która sto lat temu pozwoliła przetrwać zły czas. Czym jest dziś czyn "
           "obywatelski? Trudno bowiem było wskazać w naszej historii czas, gdy zarazem czyn "
           "obywatelski nie wiązał się z ofiarą i wyrzeczeniem, gdy nie był obciążony ryzykiem. "
           "W dodatku dziś, działając pro publico bono, można przyczyniać się do cywilizacyjnego "
           "rozwoju państwa, zwiększając jego dobrobyt. Nic bowiem nie jest dobitniejszym świadectwem "
           "solidarności niż praca dla dobra wspólnego.")

In [12]:
rouge = Rouge()

In [13]:
scores = rouge.get_scores(summary, references[0])

In [14]:
scores

[{'rouge-1': {'f': 0.3506493458030444,
   'p': 0.2983425414364641,
   'r': 0.4251968503937008},
  'rouge-2': {'f': 0.23529411280276827, 'p': 0.2, 'r': 0.2857142857142857},
  'rouge-l': {'f': 0.34166666176701393,
   'p': 0.29927007299270075,
   'r': 0.39805825242718446}}]

In [15]:
scores = scores[0]

ROUGE-N

In [16]:
scores["rouge-1"]

{'f': 0.3506493458030444, 'p': 0.2983425414364641, 'r': 0.4251968503937008}

In [17]:
scores["rouge-2"]

{'f': 0.23529411280276827, 'p': 0.2, 'r': 0.2857142857142857}

ROUGE-L

In [18]:
scores["rouge-l"]

{'f': 0.34166666176701393, 'p': 0.29927007299270075, 'r': 0.39805825242718446}

ROUGE-S

`policja zatrzymała poszukiwanych przestępców`

(policja, zatrzymała) (policja, poszukiwanych) (policja, przestępców)   
(zatrzymała, poszukiwanych) (zatrzymała, przestępców) (poszukiwanych, przestępców)

In [19]:
summaries = [summary] * len(references)

In [20]:
scores = rouge.get_scores(summaries, references)

In [21]:
scores

[{'rouge-1': {'f': 0.3506493458030444,
   'p': 0.2983425414364641,
   'r': 0.4251968503937008},
  'rouge-2': {'f': 0.23529411280276827, 'p': 0.2, 'r': 0.2857142857142857},
  'rouge-l': {'f': 0.34166666176701393,
   'p': 0.29927007299270075,
   'r': 0.39805825242718446}},
 {'rouge-1': {'f': 0.1623376574913562,
   'p': 0.13812154696132597,
   'r': 0.1968503937007874},
  'rouge-2': {'f': 0.02614378600538344,
   'p': 0.022222222222222223,
   'r': 0.031746031746031744},
  'rouge-l': {'f': 0.12903225311947208,
   'p': 0.11678832116788321,
   'r': 0.14414414414414414}},
 {'rouge-1': {'f': 0.47741934997939656,
   'p': 0.4088397790055249,
   'r': 0.5736434108527132},
  'rouge-2': {'f': 0.3896103847529095, 'p': 0.3333333333333333, 'r': 0.46875},
  'rouge-l': {'f': 0.48979591343740114,
   'p': 0.43795620437956206,
   'r': 0.5555555555555556}},
 {'rouge-1': {'f': 0.44303796978949694,
   'p': 0.3867403314917127,
   'r': 0.5185185185185185},
  'rouge-2': {'f': 0.30573247918373975,
   'p': 0.26666666

In [22]:
scores = rouge.get_scores(summaries, references, avg=True)

In [23]:
scores

{'rouge-1': {'f': 0.35174910458467945,
  'p': 0.3060773480662983,
  'r': 0.414365013501091},
 'rouge-2': {'f': 0.23014403034510345, 'p': 0.2, 'r': 0.27155052120350626},
 'rouge-l': {'f': 0.3422018976624709,
  'p': 0.3124087591240876,
  'r': 0.37967918213191587}}

In [24]:
t_summary = ("Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji "
             "nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie "
             "wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Na czym "
             "w 1999 roku polega czyn obywatelski? Pamięć o tych, którzy mimo 123 lat niewoli, upokorzeń "
             "i klęsk, nigdy nie pogodzili się z utratą wolności, a także o tych, którzy ów protest "
             "przechowali przez noc nazizmu i komunizmu na zawsze pozostanie źródłem naszej siły. "
             "Pokolenie, które czynem zbrojnym wywalczyło wolność nie zmarnowało jej, choć nie dane "
             "mu było dokończyć dzieła odbudowy polskiej państwowości. Kultura narodowa i lokalna są "
             "ze sobą splecione, mają wspólne korzenie, które określają naszą tożsamość. Wybór Krakowa "
             "na kulturalną stolicę Europy w roku 2000 potwierdza, że we współczesnym świecie docenia się "
             "udział kultury w budowaniu tożsamości, choć w przypadku tego miasta jest to na pewno bardziej "
             "kultura narodowa niż lokalna. Kraków jest dziś przykładem tego, jak działania na rzecz ochrony "
             "narodowego dziedzictwa wpływają na rozwój miasta i warunki życia jego mieszkańców. Dlatego dobre "
             "państwo trzeba i warto budować na fundamencie lokalnego dziedzictwa. Czym jest dziś czyn "
             "obywatelski? Trudno bowiem było wskazać w naszej historii czas, gdy zarazem czyn obywatelski "
             "nie wiązał się z ofiarą i wyrzeczeniem, gdy nie był obciążony ryzykiem.")

In [25]:
t_summaries = [t_summary] * len(references)

In [26]:
t_scores = rouge.get_scores(t_summaries, references, avg=True)

In [27]:
for k, v in scores.items():
    print(f"{k}: {v}")

rouge-1: {'f': 0.35174910458467945, 'p': 0.3060773480662983, 'r': 0.414365013501091}
rouge-2: {'f': 0.23014403034510345, 'p': 0.2, 'r': 0.27155052120350626}
rouge-l: {'f': 0.3422018976624709, 'p': 0.3124087591240876, 'r': 0.37967918213191587}


In [28]:
for k, v in t_scores.items():
    print(f"{k}: {v}")

rouge-1: {'f': 0.3831450366241552, 'p': 0.31538461538461543, 'r': 0.48923177020347686}
rouge-2: {'f': 0.26765156389771394, 'p': 0.22028985507246376, 'r': 0.3418180674010898}
rouge-l: {'f': 0.38080889183095556, 'p': 0.32704402515723274, 'r': 0.45759666620678374}
